# Ergodic Visualizations

## Overview
- **What**: Visualizations of core ergodic theory concepts -- time vs. ensemble averages, path-dependent wealth evolution, survivor bias, and the insurance value proposition.
- **Prerequisites**: [../core/05_ergodic_theory](../core/05_ergodic_theory.ipynb), [01_visualization_factory](01_visualization_factory.ipynb)
- **Estimated runtime**: < 2 minutes
- **Audience**: [Practitioner] / [Developer]

## Key Concepts
- **Time Average**: Growth rate experienced by a single entity over time
- **Ensemble Average**: Expected value across many parallel scenarios
- **Ergodic Divergence**: The separation between time and ensemble averages for multiplicative processes
- **Path Dependence**: How individual trajectories evolve differently despite identical statistics
- **Survivor Bias**: How averaging only surviving paths misleads decision-making

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/visualization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ergodic_insurance.visualization.technical_plots import (
    plot_ergodic_divergence,
    plot_path_dependent_wealth,
)
from ergodic_insurance.ergodic_analyzer import ErgodicAnalyzer

np.random.seed(42)

VOLATILITY = 0.2
DRIFT = 0.08
N_PATHS = 1000
MAX_YEARS = 1000

## 1. Generate Ergodic Analysis Data

Simulate geometric Brownian motion paths and compute time-average vs. ensemble-average growth rates across logarithmically-spaced time horizons.

In [ ]:
def generate_ergodic_data(n_paths=N_PATHS, max_years=MAX_YEARS,
                          volatility=VOLATILITY, drift=DRIFT):
    """Generate time/ensemble average data across time horizons."""
    time_horizons = np.unique(np.logspace(0, np.log10(max_years), 50).astype(int))
    time_averages, ensemble_averages, standard_errors = [], [], []

    # Full trajectories via GBM
    trajectories = np.ones((n_paths, max_years))
    for i in range(n_paths):
        returns = np.random.normal(drift - volatility**2 / 2, volatility, max_years)
        trajectories[i] = np.cumprod(np.exp(returns))

    for horizon in time_horizons:
        log_growth = [
            np.log(trajectories[i, horizon - 1]) / horizon
            for i in range(n_paths)
            if trajectories[i, horizon - 1] > 0
        ]
        time_averages.append(np.mean(log_growth))
        ensemble_averages.append(drift)  # Without volatility drag
        standard_errors.append(np.std(log_growth) / np.sqrt(len(log_growth)))

    return {
        "time_horizons": np.array(time_horizons),
        "time_averages": np.array(time_averages),
        "ensemble_averages": np.array(ensemble_averages),
        "standard_errors": np.array(standard_errors),
        "trajectories": trajectories,
    }


ergodic_data = generate_ergodic_data()
print(f"Generated {ergodic_data['trajectories'].shape[0]} paths "
      f"over {ergodic_data['trajectories'].shape[1]} years")
print(f"Analyzing {len(ergodic_data['time_horizons'])} time horizons")

## 2. Ergodic vs. Ensemble Divergence

The time-average growth rate includes volatility drag and diverges from the ensemble average as the horizon increases. This divergence is why insurance decisions must consider individual path dynamics rather than expected values.

In [ ]:
fig1 = plot_ergodic_divergence(
    ergodic_data["time_horizons"],
    ergodic_data["time_averages"],
    ergodic_data["ensemble_averages"],
    standard_errors=ergodic_data["standard_errors"],
    title="Ergodic vs Ensemble Average Divergence",
    figsize=(14, 8),
)
plt.show()

print("Observations:")
print("  1. Time and ensemble averages start similar but diverge")
print("  2. Time average includes volatility drag")
print("  3. This divergence justifies paying premiums above expected loss")

## 3. Volatility Impact on Divergence

Higher volatility widens the gap between time and ensemble averages, making insurance more valuable in volatile environments.

In [ ]:
# Generate scenarios with different volatilities
volatilities = {"Low Vol (10%)": 0.1, "Medium Vol (20%)": 0.2, "High Vol (30%)": 0.3}
scenarios = {}
for name, vol in volatilities.items():
    data = generate_ergodic_data(n_paths=500, max_years=500, volatility=vol)
    scenarios[name] = {
        "horizons": data["time_horizons"],
        "time_avg": data["time_averages"],
    }

fig2 = plot_ergodic_divergence(
    ergodic_data["time_horizons"][:40],
    ergodic_data["time_averages"][:40],
    ergodic_data["ensemble_averages"][:40],
    parameter_scenarios=scenarios,
    title="Volatility Impact on Ergodic Divergence",
    figsize=(14, 8),
)
plt.show()

print("Higher volatility -> greater divergence -> more insurance value")

## 4. Path-Dependent Wealth Evolution

Multiple wealth trajectories over time showing paths that hit ruin and demonstrating survivor bias. The ensemble average looks good, but many individual paths fail.

In [ ]:
def generate_wealth_trajectories(n_paths=500, n_years=100,
                                  volatility=0.25, leverage=1.5):
    """Simulate leveraged wealth trajectories with ruin."""
    trajectories = np.ones((n_paths, n_years))
    initial_wealth = 100.0
    for i in range(n_paths):
        wealth = initial_wealth
        for t in range(n_years):
            shock = np.random.normal(0.06 * leverage, volatility * leverage)
            wealth *= 1 + shock
            if wealth < initial_wealth * 0.01:
                trajectories[i, t:] = 0
                break
            trajectories[i, t] = wealth
    return trajectories


wealth_paths = generate_wealth_trajectories(n_paths=1000, n_years=100)

fig3 = plot_path_dependent_wealth(
    wealth_paths,
    ruin_threshold=1.0,
    percentiles=[5, 25, 50, 75, 95],
    highlight_ruined=True,
    add_survivor_bias_inset=True,
    title="Path-Dependent Wealth Evolution with Survivor Bias",
    figsize=(14, 9),
)
plt.show()

final_wealth = wealth_paths[:, -1]
n_ruined = np.sum(final_wealth <= 1.0)
survivor_mean = np.mean(final_wealth[final_wealth > 1.0]) if np.any(final_wealth > 1.0) else 0
total_mean = np.mean(final_wealth)

print(f"Path Statistics:")
print(f"  Total paths:           {len(wealth_paths)}")
print(f"  Ruined paths:          {n_ruined} ({n_ruined / len(wealth_paths) * 100:.1f}%)")
print(f"  Mean wealth (survivors): ${survivor_mean:.2f}")
print(f"  Mean wealth (all):       ${total_mean:.2f}")
print(f"  Survivor bias factor:    {survivor_mean / total_mean if total_mean > 0 else 0:.2f}x")

## 5. Insurance Impact on Path Evolution

Side-by-side comparison of wealth trajectories with and without insurance shows that insurance reduces extreme downside while preserving upside potential.

In [ ]:
def generate_insured_trajectories(n_paths=500, n_years=100,
                                   insurance_premium=0.02, loss_cap=0.20):
    """Compare uninsured vs. insured wealth paths."""
    uninsured = np.ones((n_paths, n_years))
    insured = np.ones((n_paths, n_years))
    initial_wealth = 100.0
    for i in range(n_paths):
        w_un, w_in = initial_wealth, initial_wealth
        for t in range(n_years):
            shock = np.random.normal(0.08, 0.25)
            # Uninsured
            w_un *= 1 + shock
            if w_un < 1.0:
                uninsured[i, t:] = 0
                w_un = 0
            else:
                uninsured[i, t] = w_un
            # Insured (capped losses, minus premium)
            capped = max(shock, -loss_cap)
            w_in *= 1 + capped - insurance_premium
            if w_in < 1.0:
                insured[i, t:] = 0
                w_in = 0
            else:
                insured[i, t] = w_in
    return {"uninsured": uninsured, "insured": insured}


comparison = generate_insured_trajectories(n_paths=800, n_years=75)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
percentiles = [5, 25, 50, 75, 95]
colors = ["#FF6B6B", "#FFA500", "#4CAF50", "#FFA500", "#FF6B6B"]

for ax, key, title in [(ax1, "uninsured", "Uninsured"), (ax2, "insured", "Insured")]:
    data = comparison[key]
    time_axis = np.arange(data.shape[1])
    pcts = np.percentile(data, percentiles, axis=0)
    for i in range(min(100, len(data))):
        survived = data[i, -1] > 1.0
        ax.plot(time_axis, data[i], color="blue" if survived else "red",
                alpha=0.02 if survived else 0.01, linewidth=0.5)
    for p, pct, c in zip(percentiles, pcts, colors):
        ax.plot(time_axis, pct, color=c, linewidth=2, label=f"{p}th pct")
    ax.set_xlabel("Time (years)")
    ax.set_ylabel("Wealth")
    ax.set_title(f"{title} Wealth Evolution")
    ax.set_yscale("log")
    ax.grid(True, alpha=0.3)
    ax.legend(loc="upper left")
    ax.axhline(y=1.0, color="red", linestyle="--", alpha=0.5)

plt.suptitle("Insurance Impact on Wealth Trajectories", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

uninsured_ruin = np.mean(comparison["uninsured"][:, -1] <= 1.0) * 100
insured_ruin = np.mean(comparison["insured"][:, -1] <= 1.0) * 100
print(f"Ruin probability -- uninsured: {uninsured_ruin:.1f}%, insured: {insured_ruin:.1f}%")
print(f"Reduction: {uninsured_ruin - insured_ruin:.1f} percentage points")

## 6. Summary: Key Insights

Four-panel summary combining the core ergodic insights.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: Time vs Ensemble Average
ax1 = axes[0, 0]
h = ergodic_data["time_horizons"][:30]
ax1.semilogx(h, ergodic_data["time_averages"][:30], "b-", lw=2, label="Time Average")
ax1.semilogx(h, ergodic_data["ensemble_averages"][:30], "r--", lw=2, label="Ensemble Average")
ax1.fill_between(
    h, ergodic_data["time_averages"][:30] - 2 * ergodic_data["standard_errors"][:30],
    ergodic_data["time_averages"][:30] + 2 * ergodic_data["standard_errors"][:30],
    alpha=0.2, color="blue",
)
ax1.set_xlabel("Time Horizon (years)")
ax1.set_ylabel("Growth Rate")
ax1.set_title("Ergodic Divergence")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Panel 2: Survival Rate Over Time
ax2 = axes[0, 1]
tp = np.arange(1, 101)
survival = [np.mean(wealth_paths[:, t - 1] > 1.0) * 100 for t in tp if t <= wealth_paths.shape[1]]
ax2.plot(tp[:len(survival)], survival, "g-", lw=2)
ax2.fill_between(tp[:len(survival)], 0, survival, alpha=0.3, color="green")
ax2.set_xlabel("Time (years)")
ax2.set_ylabel("Survival Rate (%)")
ax2.set_title("Path Survival Over Time")
ax2.grid(True, alpha=0.3)

# Panel 3: Wealth Distribution at Different Times
ax3 = axes[1, 0]
for t in [10, 25, 50, 75]:
    if t <= wealth_paths.shape[1]:
        surviving = wealth_paths[:, t - 1][wealth_paths[:, t - 1] > 1.0]
        if len(surviving) > 0:
            ax3.hist(np.log10(surviving), bins=30, alpha=0.5,
                     label=f"Year {t}", density=True)
ax3.set_xlabel("Log10(Wealth)")
ax3.set_ylabel("Probability Density")
ax3.set_title("Wealth Distribution Evolution")
ax3.legend()
ax3.grid(True, alpha=0.3)

# Panel 4: Insurance Value vs Volatility
ax4 = axes[1, 1]
vols = np.linspace(0.05, 0.40, 50)
ins_value = vols**2 / 2  # Approximation
ax4.plot(vols * 100, ins_value * 100, "purple", lw=2)
ax4.fill_between(vols * 100, 0, ins_value * 100, alpha=0.3, color="purple")
ax4.set_xlabel("Volatility (%)")
ax4.set_ylabel("Insurance Value (% per year)")
ax4.set_title("Insurance Value vs Volatility")
ax4.grid(True, alpha=0.3)

plt.suptitle("Ergodic Insurance Framework: Key Insights", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## Key Takeaways

- Time averages diverge from ensemble averages for multiplicative processes; this gap grows with the time horizon.
- Many paths lead to ruin despite positive expected returns -- survivor bias inflates perceived performance.
- Insurance improves time-average growth; its value increases with volatility.
- Optimal insurance premiums can exceed expected losses because they address path-level survival, not ensemble expectations.

## Next Steps

- [04_convergence_monitoring](04_convergence_monitoring.ipynb) -- convergence diagnostics for simulation validation
- [05_ruin_analysis_plots](05_ruin_analysis_plots.ipynb) -- ruin cliff and ROE-ruin frontier deep dive
- [../core/05_ergodic_theory](../core/05_ergodic_theory.ipynb) -- mathematical foundations